In [1]:
import numpy as np
from time import time
from pynq import Overlay
from pynq import allocate

if __name__ == "__main__":
    ol = Overlay("/home/xilinx/jupyter_notebooks/R2plus1D_onelayer/test_onelayer.bit")
    ip_r2plus1d = ol.r2plus1d_0
    
    print("IP loaded!")

IP loaded!


In [2]:
    # calculate the number of input data
    INPUT = open("/home/xilinx/jupyter_notebooks/R2plus1D_onelayer/input.dat", "r+")
    inputNum = 0
    line = INPUT.readline()
    while line:
        inputNum = inputNum + 1
        line = INPUT.readline()
    
    print("inputNum:", inputNum)

    # calculate the number of kernel1 weight
    KERNEL1 = open("/home/xilinx/jupyter_notebooks/R2plus1D_onelayer/Conv3d1weight.dat", "r+")
    kernel1Num = 0
    line = KERNEL1.readline()
    while line:
        kernel1Num = kernel1Num + 1
        line = KERNEL1.readline()
    
    print("kernel1Num:", kernel1Num)
    
    # calculate the number of kernel2 weight
    KERNEL2 = open("/home/xilinx/jupyter_notebooks/R2plus1D_onelayer/Conv3d2weight.dat", "r+")
    kernel2Num = 0
    line = KERNEL2.readline()
    while line:
        kernel2Num = kernel2Num + 1
        line = KERNEL2.readline()
    
    print("kernel2Num:", kernel2Num)
    
    # allocate memory 
    inputBuffer = allocate(shape=(inputNum,), dtype=np.single)
    outputBuffer = allocate(shape=(200704,), dtype=np.single)
    kernel1Buffer = allocate(shape=(kernel1Num,), dtype=np.single)
    kernel2Buffer = allocate(shape=(kernel2Num,), dtype=np.single)

    # load input data
    print("loading input data...")
    INPUT.seek(0)
    for i in range(inputNum):
        line = INPUT.readline()
        inputBuffer[i] = line
        if (i % 1000 == 0):
            print(i, "/", inputNum)
    INPUT.close()
    
    # load kernel1 weight
    print("loading kernel1 data...")
    KERNEL1.seek(0)
    for i in range(kernel1Num):
        line = KERNEL1.readline()
        kernel1Buffer[i] = line
        if (i % 1000 == 0):
            print(i, "/", kernel1Num)
    KERNEL1.close()
    
    # load kernel2 weight
    print("loading kernel2 data...")
    KERNEL2.seek(0)
    for i in range(kernel2Num):
        line = KERNEL2.readline()
        kernel2Buffer[i] = line
        if (i % 1000 == 0):
            print(i, "/", kernel2Num)
    KERNEL2.close()

inputNum: 36963
kernel1Num: 6615
kernel2Num: 8640
loading input data...
0 / 36963
1000 / 36963
2000 / 36963
3000 / 36963
4000 / 36963
5000 / 36963
6000 / 36963
7000 / 36963
8000 / 36963
9000 / 36963
10000 / 36963
11000 / 36963
12000 / 36963
13000 / 36963
14000 / 36963
15000 / 36963
16000 / 36963
17000 / 36963
18000 / 36963
19000 / 36963
20000 / 36963
21000 / 36963
22000 / 36963
23000 / 36963
24000 / 36963
25000 / 36963
26000 / 36963
27000 / 36963
28000 / 36963
29000 / 36963
30000 / 36963
31000 / 36963
32000 / 36963
33000 / 36963
34000 / 36963
35000 / 36963
36000 / 36963
loading kernel1 data...
0 / 6615
1000 / 6615
2000 / 6615
3000 / 6615
4000 / 6615
5000 / 6615
6000 / 6615
loading kernel2 data...
0 / 8640
1000 / 8640
2000 / 8640
3000 / 8640
4000 / 8640
5000 / 8640
6000 / 8640
7000 / 8640
8000 / 8640


In [3]:
    # start IP
    
    # write input, output, kernel1 weight, kernel2 weight address
    ip_r2plus1d.write(0x10, inputBuffer.device_address)
    ip_r2plus1d.write(0x1C, outputBuffer.device_address)
    ip_r2plus1d.write(0x28, kernel1Buffer.device_address)
    ip_r2plus1d.write(0x34, kernel2Buffer.device_address)
        
    # ap_start
    print("IP start")
    
    t1 = time()
    
    ip_r2plus1d.write(0x00, 0x01)
    # wait for the computation to finish
    while (ip_r2plus1d.read(0x00) & 0x4) == 0x0:
        continue
        
    t2 = time()
        
    print("IP done")
    print("time cost:", t2-t1)

IP start
IP done
time cost: 12.853078126907349


In [4]:
    OUTPUT = open("/home/xilinx/jupyter_notebooks/R2plus1D_onelayer/ReLU2output.dat", "r+")
    fout_0001 = open("/home/xilinx/jupyter_notebooks/R2plus1D_onelayer/out_0001.dat", "w")
    fout_001  = open("/home/xilinx/jupyter_notebooks/R2plus1D_onelayer/out_001.dat", "w")
    fout_01   = open("/home/xilinx/jupyter_notebooks/R2plus1D_onelayer/out_01.dat", "w")

    print("This file record the results with error >= 0.0001", file=fout_0001)
    print("====================", file=fout_0001)
    print("This file record the results with error >= 0.001", file=fout_001)
    print("====================", file=fout_001)
    print("This file record the results with error >= 0.01", file=fout_01)
    print("====================", file=fout_01)
    
    OUTPUT.seek(0)
    for i in range(200704):
        line = np.single(OUTPUT.readline())
        err = (outputBuffer[i] - line) / line
        if (outputBuffer[i] != line):
            if (err >= 0.0001):
                print("[ERROR]: index", i, ", result:", outputBuffer[i], ", gold:", line, ", error:", err, file=fout_0001)
                if (err >= 0.001):
                    print("[ERROR]: index", i, ", result:", outputBuffer[i], ", gold:", line, ", error:", err, file=fout_001)
                    if (err >= 0.01):
                        print("[ERROR]: index", i, ", result:", outputBuffer[i], ", gold:", line, ", error:", err, file=fout_01)
            
    fout_0001.close()
    fout_001.close()
    fout_01.close()
    OUTPUT.close()

<ipython-input-4-56af143787e0>:16: RuntimeWarning: invalid value encountered in double_scalars
  err = (outputBuffer[i] - line) / line
<ipython-input-4-56af143787e0>:16: RuntimeWarning: divide by zero encountered in double_scalars
  err = (outputBuffer[i] - line) / line
